# Extracción de Obligaciones desde Resoluciones de Calificación Ambiental

Este script forma parte de un sistema diseñado para extraer, procesar y categorizar obligaciones ambientales contenidas en documentos de Resoluciones de Calificación Ambiental (RCA) emitidos por el Servicio de Evaluación Ambiental de Chile.

### Librerías

In [ ]:
# Librerías estándar de Python
import copy
import json
import os
import re
import time
import traceback

# Librerías de terceros
from bs4 import BeautifulSoup
import numpy as np
import openpyxl
import pandas as pd
import pdfplumber
from transformers import AutoTokenizer

# Librerías de entorno y AI
from dotenv import load_dotenv
import openai
from openai import OpenAI
from langchain_openai import OpenAI

# Pydantic
from enum import Enum
from pydantic import BaseModel, Field, constr, conlist, ValidationError
from typing import List, Optional, Literal, Annotated

### Establecer Directorios

In [ ]:
sector = "energia"
direccion_output = os.path.join(os.environ['USERPROFILE'], "RUTA", f"archivos_{sector}")
load_dotenv()
api_key_openai = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(
    api_key=api_key_openai
)

### Instrucciones

In [ ]:
multiple = {
    "categorizar": f"""
    Contexto:  
    Se te entregará información original de tablas de Resoluciones de Calificación Ambiental (RCA) del Servicio de Evaluación Ambiental de Chile, relativas a la evaluación de un proyecto.
    
    Objetivo:  
    De cada tabla (cada fila), extrae y desglosa las obligaciones individuales exigidas al titular del proyecto (la empresa). Cada fila contiene múltiples obligaciones y debes separarlas adecuadamente.
    
    - Las obligaciones emergen siempre del texto de "contenido_variable", muchas veces están enumeradas. Las otras variables son de apoyo. 
    - Debes entender el sentido de cada obligación, no limitarte a un corte mecánico.
    - No repitas nunca una misma obligación.
    - No incluyas información que no sea una obligación explícita (no usar meras explicaciones o contexto).
    - Si en una misma fila hay varias acciones o requisitos diferentes, debes dividirlos en obligaciones separadas (una por fila).
    
    Instrucciones:  
    Por cada fila del DataFrame original:
    
    1. Crea un nuevo DataFrame donde cada obligación individual sea una fila.
    2. Incluye las siguientes variables en el nuevo DataFrame:
       
       1. **"cell", "seccion", "numero_tabla"**: Se extraen tal cual del input. Identifican el documento, la sección y la tabla.
       
       2. **"resumen"**: Un breve resumen que describa claramente la obligación.
       
       3. **"componente_materia"**: Categoriza según el componente ambiental protegido.
          - Si se menciona alguno de los siguientes decretos, asignar a: 
                  - D.S. 148 Minsal → "residuos peligrosos"
                  - D.S. 43 o 57 Minsal "sustancias peligrosas"
                  - D.S. 189 Minsal  "residuos sólidos"
                  - D.S. 351 MOP "residuos líquidos"
          - Elige la categoría más cercana si no es clara.  
          - Ejemplo: "Emisiones de CO2" →  "aire" / "protección de animales" → "fauna".  
          - Importante: Todas las obligaciones del mismo "numero_tabla" deben tener el mismo "componente_materia".
       
       4. **"fase"**: Indica la fase del proyecto a la que aplica la obligación (construcción, operación, cierre, u otra).  
          - Si se indica "Todos", "durante la vida útil", etc., usa ["construcción", "operación", "cierre"].
       
       5. **"fuente" y "numero_fuente"**:  
          - Identifica las fuentes legales/reglamentarias mencionadas.  
          - "fuente": Lista con las fuentes (ej. "ley", "decreto con fuerza de ley"). Cambia abreviaturas a su forma completa (D.S. = decreto supremo, R.Ex. = resolución).  
          - "numero_fuente": Identificadores correspondientes, manteniendo el orden.  
          - Si no se mencionan normas y "normas" está vacío, usa "fuente" = ["decisión de autoridad"] y "numero_fuente" = [""].
          - Si "seccion" = "compromisos_voluntarios", usa "fuente" = ["compromiso ambiental voluntario"] y "numero_fuente" = [""].
       
       6. **"objeto"**: Categoriza el objetivo o propósito de la obligación. Se deben priorizar otras categorías por sobre "Cumplimiento de Normas Ambientales", y esta categoría se prioriza por sobre "Otros". 
       
       7. **"orden_obligacion"**: "principal" o "secundaria".  
          - "Principal": Describe una acción o medida central.  
          - "Secundaria": Busca asegurar, reportar, informar o probar el cumplimiento de otra obligación principal (ejs: control interno, seguimiento, reporte, registro, informes, apoyo, fotografías).
       
       8. **"tipo_obligación"**: "Obligación de Medios" o "Obligación de Resultados".  
          - Medios: Estándares o conductas a seguir para lograr un resultado.  
          - Resultados: Exigen un resultado específico (ej. umbrales de emisiones) sin indicar cómo alcanzarlo.
       
       9. **"justificacion"**: Extracto fiel del texto original de donde se desprende la obligación.
       
       10. **"frecuencia"**: Clasifica la frecuencia de la obligación.  
           - Si es anual, semestral, mensual u otra periodicidad, etiquetar como "periódica".
       
    Sigue los ejemplos provistos como guía para procesar nuevas entradas.  
    Recuerda: La separación de las obligaciones es crucial. Cada obligación debe ser una fila independiente.
    """  , 

    "eg_input_1": """
        [
            {
            "cell": "B141",
            "seccion": "mitigacion",
            "numero_tabla": "7.2.1",
            "pas": "",
            "normas": "",
            "componente_materia": "Emisiones de Partículas",
            "fase": "Cierre",
            "parte_asociada": "",
            "riesgo_contingencia": "",
            "mitigacion": "",
            "impacto": "", 
            "nombre_variable": "lugar_forma_oportunidad", 
            "contenido_variable": "Lugar: Cubeta del depósito de relaves. Forma: Rociar la cubierta de la cubeta del DR con agua de mar o similares características. Oportunidad: La inspección visual del estado de la costra salina y el rociado será de 5 años luego del cierre del proyecto..",
            },
            {
            "cell": "B141",
            "seccion": "mitigacion",
            "numero_tabla": "7.2.1",
            "pas": "",
            "normas": "",
            "componente_materia": "Emisiones de Partículas",
            "fase": "Cierre",
            "parte_asociada": "",
            "riesgo_contingencia": "",
            "mitigacion": "",
            "impacto": "", 
            "nombre_variable": "objetivo_descripcion_justificacion", 
            "contenido_variable": "Objetivo: Minimizar las emisiones de material particulado por erosión eólica del DR luego del cierre del proyecto. Descripción: Luego del cese de la operación, se realizará el monitoreo visual posterior a un evento de lluvia de gran intensidad y luego, en caso de ser necesario, rociar la cubierta de la cubeta del DR con agua de mar o similares características en lugares que se visualicen afectados como resultado del monitoreo visual y así favorecer la formación de nueva “costra salina”. Para esto, se estima un requerimiento de 1 litro/m2 de agua de mar o salmuera de descarte de osmosis inversa, con una concentración de sales de 35 g/L, de forma de favorecer la formación de la costra salina en caso de que se vea deteriorada por una laguna temporal de aguas. Justificación: Minimizar las emisiones de material particulado por erosión eólica del DR luego del cierre del proyecto.",
            },
            {
            "cell": "B141",
            "seccion": "mitigacion",
            "numero_tabla": "7.2.1",
            "pas": "",
            "normas": "",
            "componente_materia": "Emisiones de Partículas",
            "fase": "Cierre",
            "parte_asociada": "",
            "riesgo_contingencia": "",
            "mitigacion": "",
            "impacto": "", 
            "nombre_variable": "indicador_cumplimiento", 
            "contenido_variable": "Se mantendrán informes y registros periódicos de las actividades de monitoreo y control de la costra salina durante la fase de cierre",
            }
        ]
        """,
    "eg_output_1": """
        [
            {
            "cell": "B141",
            "seccion": "mitigacion",
            "numero_tabla": "7.2.1",
            "resumen": "se realizará el monitoreo visual posterior a un evento de lluvia de gran intensidad y luego, en caso de ser necesario, rociar la cubierta de la cubeta del DR con agua de mar o similares características en lugares que se visualicen afectados como resultado del monitoreo visual y así favorecer la formación de nueva “costra salina”.",
            "componente_materia": "aire",
            "fase": ["cierre"],
            "fuente": ["decisión de autoridad"],
            "numero_fuente": [""],
            "objeto": "mitigación, compensación o reparación de efectos no deseados",
            "orden_obligacion": "Principal",
            "tipo_obligacion": "Obligación de medios",
            "justificacion": "Luego del cese de la operación, se realizará el monitoreo visual posterior a un evento de lluvia de gran intensidad y luego, en caso de ser necesario, rociar la cubierta de la cubeta del DR con agua de mar o similares características en lugares que se visualicen afectados como resultado del monitoreo visual y así favorecer la formación de nueva “costra salina”. Para esto, se estima un requerimiento de 1 litro/m2 de agua de mar o salmuera de descarte de osmosis inversa, con una concentración de sales de 35 g/L, de forma de favorecer la formación de la costra salina en caso de que se vea deteriorada por una laguna temporal de aguas.",
            "frecuencia": "eventual"
          },
          {
            "cell": "B141",
            "seccion": "mitigacion",
            "numero_tabla": "7.2.1",
            "resumen": "informes y registros periódicos de las actividades de monitoreo y control de la costra salina",
            "componente_materia": "aire",
            "fase": ["cierre"],
            "fuente": ["decisión de autoridad"],
            "numero_fuente": [""],
            "objeto": "monitoreo, mantención, seguimiento y entrega de información", 
            "orden_obligacion": "Secundaria",
            "tipo_obligacion": "Obligación de medios",
            "justificacion": "Se mantendrán informes y registros periódicos de las actividades de monitoreo y control de la costra salina durante la fase de cierre",
            "frecuencia": "periódica"
          }
          ]
        """, 
    "eg_input_2": """
        [
           {
            "cell": "B12",
            "seccion": "forma_cumplimiento",
            "numero_tabla": "7.1",
            "pas": "",
            "normas": "Decreto Supremo N° 405 y Ley 19.300",
            "fase": "Construcción, Operación y Cierre",
            "parte_asociada": "Toda la obra",
            "componente_materia": "Emisiones de Material Particulado",
            "acciones_contingencia": "",
            "acciones_emergencia": "",
            "riesgo_contingencia": "",
            "mitigacion": "",
            "impacto": "",
            "nombre_variable": "indicador_cumplimiento", 
            "contenido_variable": "El indicador para contolar el cumplimiento será: - registro fotográfico mensual de la humectación - visita anual del regulador",
           },
           {
            "cell": "B12",
            "seccion": "forma_cumplimiento",
            "numero_tabla": "7.1",
            "pas": "",
            "normas": "Decreto Supremo N° 405 y Ley 19.300",
            "fase": "Construcción, Operación y Cierre",
            "parte_asociada": "Toda la obra",
            "componente_materia": "Emisiones de Material Particulado",
            "acciones_contingencia": "",
            "acciones_emergencia": "",
            "riesgo_contingencia": "",
            "mitigacion": "",
            "impacto": "",
            "nombre_variable": "forma_cumplimiento", 
            "contenido_variable": "Para controlar las emisiones atmosféricas, el Titular aplicará las siguientes acciones de control: Humectación de frentes de trabajo. Aplicación de supresor de polvo en 6 ha de suelo basal del DRF mensualmente. Exigencia de revisión técnica al día de vehículos del proyecto.",
            }
        ]
        """,
    "eg_output_2": """
        [
          {
            "cell": "B12",
            "seccion": "forma_cumplimiento",
            "numero_tabla": "7.1",
            "resumen": "Aplicar humectación de frentes de trabajo para controlar emisiones atmosféricas",
            "componente_materia": "aire",
            "fase": ["construcción", "operación", "cierre"],
            "fuente": ["decreto supremo", "ley"],
            "numero_fuente": ["405", "19300"],
            "objeto": "prevención ambiental",
            "orden_obligacion": "Principal",
            "tipo_obligacion": "Obligación de medios",
            "justificacion": "Para controlar las emisiones atmosféricas, el Titular aplicará las siguientes acciones de control: Humectación de frentes de trabajo.",
            "frecuencia": "permanente"
          },
          {
            "cell": "B12",
            "seccion": "forma_cumplimiento",
            "numero_tabla": "7.1",
            "resumen": "Aplicar mensualmente supresor de polvo en 6 ha de suelo basal del DRF",
            "componente_materia": "aire",
            "fase": ["construcción", "operación", "cierre"],
            "fuente": ["decreto supremo", "ley"],
            "numero_fuente": ["405", "19300"],
            "objeto": "prevención ambiental",
            "orden_obligacion": "Secundaria",
            "tipo_obligacion": "Obligación de medios",
            "justificacion": "Aplicación de supresor de polvo en 6 ha de suelo basal del DRF mensualmente.",
            "frecuencia": "periódica"
          },
          {
            "cell": "B12",
            "seccion": "forma_cumplimiento",
            "numero_tabla": "7.1",
            "resumen": "Exigir revisión técnica al día de vehículos del proyecto",
            "componente_materia": "aire",
            "fase": ["construcción", "operación", "cierre"],
            "fuente": ["decreto supremo", "ley"],
            "numero_fuente": ["405", "19300"],
            "objeto": "prevención ambiental",
            "orden_obligacion": "Secundaria",
            "tipo_obligacion": "Obligación de medios",
            "justificacion": "Exigencia de revisión técnica al día de vehículos del proyecto.",
            "frecuencia": "cumplimiento de permisos"
          }, 
          {
            "cell": "B12",
            "seccion": "forma_cumplimiento",
            "numero_tabla": "7.1",
            "resumen": "registro fotográfico mensual de la humectación",
            "componente_materia": "aire",
            "fase": ["construcción", "operación", "cierre"],
            "fuente": ["decreto supremo", "ley"],
            "numero_fuente": ["405", "19300"],
            "objeto": "monitoreo, mantención, seguimiento y entrega de información",
            "orden_obligacion": "Secundaria",
            "tipo_obligacion": "Obligación de medios",
            "justificacion": "- registro fotográfico mensual de la humectación",
            "frecuencia": "periódica"
          },
           {
            "cell": "B12",
            "seccion": "forma_cumplimiento",
            "numero_tabla": "7.1",
            "resumen": "visita anual del regulador",
            "componente_materia": "aire",
            "fase": ["construcción", "operación", "cierre"],
            "fuente": ["decreto supremo", "ley"],
            "numero_fuente": ["405", "19300"],
            "objeto": "monitoreo, mantención, seguimiento y entrega de información",
            "orden_obligacion": "Secundaria",
            "tipo_obligacion": "Obligación de medios",
            "justificacion": "- visita anual del regulador",
            "frecuencia": "periódica"
          }
          ] 
          """
}

In [ ]:
unico = {
    "categorizar": f"""
        Objetivo:
        El objetivo es categorizar las filas presentes en el archivo de entrada organizadas bajo diferentes 'numeros_tabla'.
        Por cada fila de entrada debe generarse una fila de salida, con las siguientes dimensiones:
                
        1. "cell", "seccion", "numero_tabla": Identificador único del documento revisado, la sección del documento y el identificador único de la tabla de dicha obligación.
    
        3. "resumen": Proporcionar un breve resumen que describa la tabla.
        
        4. "fase": Identificar si aplica a fases de construcción, operación, cierre u otra.
           - Si la fase es 'Todos', 'durante la vida útil del proyecto', o similar, categorizar como ['construcción', 'operación', 'cierre'].
        
        5. "fuente" y "numero_fuente": Extrae y categoriza las fuentes legales o reglamentarias mencionadas.
           - "fuente": Listar las fuentes mencionadas (por ejemplo, 'ley', 'decreto supremo'). Abreviaciones como 'D.S.' deben categorizarse como 'decreto supremo', 'R.Ex.' como 'resolución'.
           - "numero_fuente": Incluir los identificadores correspondientes, manteniendo una longitud de la lista igual a la lista de fuentes.
            - Si "seccion" es "pas"; "numero_fuente" Siempre es el número del PAS (que siempre es un número de entre 111 y 160) y "fuente" es ["permiso ambiental sectorial"]
            - Si no se mencionan normas específicas, agrega a "fuente" ["decisión de autoridad"] y a "numero_fuente" [""].
            
        6. "objeto": Categoriza el objetivo o propósito perseguido. Se deben priorizar otras categorías por sobre "Cumplimiento de Normas Ambientales", y esta categoría se prioriza por sobre "Otros". 
    
        7. "tipo_obligación": Determinar si es una obligación de medios o de resultados.
           - 'Obligación de Medios': Impone el cumplimiento de un estándar específico de conducta a seguir para alcanzar un resultado.
           - 'Obligación de Resultados': Exige el cumplimiento de un resultado específico, como umbrales de emisiones, sin especificar el medio para su cumplimiento.
            
        8. "sin_condiciones": Variable booleana que toma valor True cuando 'condiciones' es 'no hay', 'no aplica' o algo similar, y en caso contrario toma valor False. 
        
        9. "frecuencia": Categoriza la frecuencia de la obligación.
            - Si debe cumplirse "anual", "semestral", "mensual" o similar, categórizala como "periódica".
            - Si "seccion" es "contingencias_emergencias", "frecuencia" debe ser "eventual".
    
        Detalles Finales
        - Si hay variables de entrada que no tienen texto, es irrelevante, debes cumplir tu objetivo igual. 
        - Si alguna categoría no es clara, elige la opción más cercana temática y semánticamente.
        - Recuerda, siempre usa una fila de entrada por una fila de salida, reflejando con precisión la intención y el contexto del texto original. 
        """, 
    "eg_input_1": """
        [
            {
            "cell": "B12",
            "seccion": "pas",
            "numero_tabla": "6.1.1",
            "pas": "151",
            "normas": "",
            "fase": "Durante la Fase de Construcción",
            "parte_asociada": "Permiso para la corta, destrucción o descepado de formaciones xerofíticas según se establece en el artículo 151 del Reglamento del SEIA.",
            "componente_materia": "",
            "mitigacion": "",
            "impacto": "",
            "nombre_variable": "condiciones",
            "contenido_variable": "Condiciones: a) El titular deberá actualizar la información de la cobertura que poseen las especies suculentas columnares. b) Cultivar, y posteriormente proteger durante todo el proyecto, los ejemplares de la especie Bridgesia incisifolia en el sector Nº 5, con una superficie de 0,4 hectáreas de protección buffer de la quebrada Las Mollacas.", 
            }
        ]
        """, 
    "eg_output_1": """
        [
          {
            "cell": "B12",
            "seccion": "pas",
            "numero_tabla": "6.1.1",
            "titulo": "Permiso para la corta, destrucción o descepado de formaciones xerofíticas según se establece en el artículo 151 del Reglamento del SEIA.",
            "fase": ["construcción"],
            "fuente": ["permiso ambiental sectorial"],
            "numero_fuente": ["203"],
            "objeto": "monitoreo, mantención, seguimiento y entrega de información",
            "sin_condiciones": "False",
            "frecuencia": "cumplimiento de permisos"
          }
          ]
          """, 
        "eg_input_2": """
        [
            {
            "cell": "B12",
            "seccion": "pas",
            "numero_tabla": "6.1.2",
            "pas": "203",
            "normas": "",
            "fase": "Fase de Operación",
            "parte_asociada": "La Dirección General de Aguas, Región de Coquimbo, mediante Ordinario N°22 de fecha 14 de enero de 2021, se pronunció conforme respecto de los requisitos para el otorgamiento del presente permiso.",
            "componente_materia": "",
            "mitigacion": "",
            "impacto": "",
            "nombre_variable": "condiciones",
            "contenido_variable":"No aplica",            
            }
        ]
        """, 
    "eg_output_2": """
        [
          {
            "cell": "B12",
            "seccion": "pas",
            "numero_tabla": "6.1.2",
            "titulo": "En la fase de operación, la Dirección General de Aguas de la Región de Coquimbo otorgó el permiso correspondiente mediante el Ordinario N°22 del 14 de enero de 2021",
            "fase": ["operación"],
            "fuente": ["permiso ambiental sectorial"],
            "numero_fuente": ["151"],
            "objeto": "monitoreo, mantención, seguimiento y entrega de información",
            "sin_condiciones": "True",
            "frecuencia": "cumplimiento de permisos"
          }
          ]
        """
}

In [ ]:
duplicacion_multiple = {
    "instruccion": f"""
    ## Contexto:
    Te proporcionaré un archivo JSON que contiene un conjunto de filas, cada una describiendo obligaciones presentes en una Resolución de Calificación Ambiental (RCA) de un proyecto de inversión en Chile.

    ## Objetivo 1:
    Tu tarea es eliminar las filas duplicadas, es decir, las filas que contienen una misma obligación. 
    - Es posible que existan diferencias semánticas en la manera en la que una misma obligación es expresada. Si ello ocurre, deberás combinar todas las filas que tengan la misma obligación (pueden ser más de 2 incluso) para formar una fila completa y coherente. 
    - El JSON resultante deberá cumplir con los formatos especificados para cada variable, y cada fila deberá estar ordenada de acuerdo con la estructura determinada. 
    - El resultado final siempre deberá contar con un número igual o menor de filas que el recibido. Nunca se deberá entregar una tabla vacía como resultado. 
    - Debes preservar la máxima fidelidad al texto original. Si una de las filas duplicadas contiene parte del contenido y la otra tiene otra porción, debes combinarlas sin repetir el contenido.
    - Si No estás seguro de si una fila es un duplicado, es preferible eliminarla y combinarla con otra. 

    ## Objetivo 2: 
    Asegurar consistencia: 
    - Las variables de "componente_materia", "fuente", "numero_fuente" SIEMPRE deben tener el mismo valor para todas las filas entregadas, si una fila no cumple, debes forzar para que tenga lo mismo que la mayoría de las filas. 
        - Ejemplo: 3 obligaciones de "fauna" y 1 de "flora": reemplazar "flora" por "fauna". 
    - Si hay alguna obligación "Secundaria" entonces debe existir alguna obligación "Principal", según lo que estimes más adecuado. 
    Te proporcionaré algunos ejemplos de entradas y salidas esperadas. Úsalos como guía para procesar nuevas entradas.

    Debes evitar a toda costa tener obligaciones duplicadas, esa es tu prioridad!
        """,
    "eg_input_1": """
          {
            "cell": "B141",
            "seccion": "mitigacion",
            "numero_tabla": "7.2.1",
            "resumen": "informes y registros periódicos de las actividades de monitoreo y control de la costra salina",
            "componente_materia": "aire",
            "fase": ["cierre"],
            "fuente": ["decisión de autoridad"],
            "numero_fuente": [""],
            "objeto": "monitoreo, mantención, seguimiento y entrega de información", 
            "orden_obligacion": "Secundaria",
            "tipo_obligacion": "Obligación de medios",
            "justificacion": "Se mantendrán informes y registros periódicos de las actividades de monitoreo y control de la costra salina durante la fase de cierre",
            "frecuencia": "periódica"
          },
          {
            "cell": "B141",
            "seccion": "mitigacion",
            "numero_tabla": "7.2.1",
            "resumen": "se realizará el monitoreo visual posterior a un evento de lluvia de gran intensidad y luego, en caso de ser necesario, rociar la cubierta de la cubeta del DR con agua de mar o similares características en lugares que se visualicen afectados como resultado del monitoreo visual y así favorecer la formación de nueva “costra salina”.",
            "componente_materia": "aire",
            "fase": ["cierre"],
            "fuente": ["decisión de autoridad"],
            "numero_fuente": [""],
            "objeto": "mitigación, compensación o reparación de efectos no deseados",
            "orden_obligacion": "Secundaria",
            "tipo_obligacion": "Obligación de medios",
            "justificacion": "Luego del cese de la operación, se realizará el monitoreo visual posterior a un evento de lluvia de gran intensidad y luego, en caso de ser necesario, rociar la cubierta de la cubeta del DR con agua de mar o similares características en lugares que se visualicen afectados como resultado del monitoreo visual y así favorecer la formación de nueva “costra salina”. Para esto, se estima un requerimiento de 1 litro/m2 de agua de mar o salmuera de descarte de osmosis inversa, con una concentración de sales de 35 g/L, de forma de favorecer la formación de la costra salina en caso de que se vea deteriorada por una laguna temporal de aguas.",
            "frecuencia": "eventual"
          },
          {
            "cell": "B141",
            "seccion": "mitigacion",
            "numero_tabla": "7.2.1",
            "resumen": "informe y registros periódicos de las actividades de monitoreo y control de la costra salina",
            "componente_materia": "aire",
            "fase": ["cierre"],
            "fuente": [""],
            "numero_fuente": [""],
            "objeto": "monitoreo, mantención, seguimiento y entrega de información", 
            "orden_obligacion": "Secundaria",
            "tipo_obligacion": "Obligación de medios",
            "justificacion": "Se mantendrán informes de las actividades de monitoreo y control de la costra salina durante la fase de cierre",
            "frecuencia": "periódica"
          }
        """, 
    "eg_output_1": """ 
          {
            "cell": "B141",
            "seccion": "mitigacion",
            "numero_tabla": "7.2.1",
            "resumen": "informes y registros periódicos de las actividades de monitoreo y control de la costra salina",
            "componente_materia": "aire",
            "fase": ["cierre"],
            "fuente": ["decisión de autoridad"],
            "numero_fuente": [""],
            "objeto": "monitoreo, mantención, seguimiento y entrega de información", 
            "orden_obligacion": "Secundaria",
            "tipo_obligacion": "Obligación de medios",
            "justificacion": "Se mantendrán informes y registros periódicos de las actividades de monitoreo y control de la costra salina durante la fase de cierre",
            "frecuencia": "periódica"
          },
          {
            "cell": "B141",
            "seccion": "mitigacion",
            "numero_tabla": "7.2.1",
            "resumen": "se realizará el monitoreo visual posterior a un evento de lluvia de gran intensidad y luego, en caso de ser necesario, rociar la cubierta de la cubeta del DR con agua de mar o similares características en lugares que se visualicen afectados como resultado del monitoreo visual y así favorecer la formación de nueva “costra salina”.",
            "componente_materia": "aire",
            "fase": ["cierre"],
            "fuente": ["decisión de autoridad"],
            "numero_fuente": [""],
            "objeto": "mitigación, compensación o reparación de efectos no deseados",
            "orden_obligacion": "Principal",
            "tipo_obligacion": "Obligación de medios",
            "justificacion": "Luego del cese de la operación, se realizará el monitoreo visual posterior a un evento de lluvia de gran intensidad y luego, en caso de ser necesario, rociar la cubierta de la cubeta del DR con agua de mar o similares características en lugares que se visualicen afectados como resultado del monitoreo visual y así favorecer la formación de nueva “costra salina”. Para esto, se estima un requerimiento de 1 litro/m2 de agua de mar o salmuera de descarte de osmosis inversa, con una concentración de sales de 35 g/L, de forma de favorecer la formación de la costra salina en caso de que se vea deteriorada por una laguna temporal de aguas.",
            "frecuencia": "eventual"
          }        
    """
}

duplicacion_unico = {
    "instruccion": f"""
    ## Contexto:
    Te proporcionaré un archivo JSON que contiene un conjunto de filas, cada una describiendo obligaciones presentes en una Resolución de Calificación Ambiental (RCA) de un proyecto de inversión en Chile.

    ## Objetivo:
    Tu tarea es eliminar las filas duplicadas, es decir, las filas con el mismo valor en la variable "numero_tabla" o filas con distinto "numero_tabla" pero el mismo contenido. 
    - Si hay diferencias en los valores de estas filas duplicadas, deberás combinarlas para formar una fila completa y coherente. El JSON resultante deberá cumplir con los formatos especificados para cada variable, y cada fila deberá estar ordenada de acuerdo con la estructura de la clase `FINAL`. 
    - El resultado final siempre deberá tener una sola fila por cada "numero_tabla".
    - Debes preservar la máxima fidelidad al texto original. Si una de las filas duplicadas contiene parte del contenido y la otra tiene otra porción, debes combinarlas sin repetir el contenido.
    - Si una variable está vacía en una fila duplicada, utiliza el contenido disponible en la otra fila. Si ambas están vacías, mantén el campo vacío. 
    - Las variable "seccion" debe mantenerse exactamente como está en las filas originales.
    - Las filas que no tienen duplicados deben mantenerse inalteradas.
    Te proporcionaré algunos ejemplos de entradas y salidas esperadas. Úsalos como guía para procesar nuevas entradas.
    Debes evitar a toda costa tener obligaciones duplicadas, esa es tu prioridad!
    Debes entregar Sólo 1 Fila Siempre
        """,
    "eg_input_1": """
          {
            "cell": "B12",
            "seccion": "pas",
            "numero_tabla": "6.1.1",
            "titulo": "En la fase de operación, la Dirección General de Aguas de la Región de Coquimbo otorgó el permiso correspondiente mediante el Ordinario N°22 del 14 de enero de 2021",
            "fase": ["operación"],
            "fuente": ["permiso ambiental sectorial"],
            "numero_fuente": ["151"],
            "objeto": "monitoreo, mantención, seguimiento y entrega de información",
            "sin_condiciones": "True",
            "frecuencia": "cumplimiento de permisos"
          },
          {
            "cell": "B12",
            "seccion": "pas",
            "numero_tabla": "6.1.1",
            "titulo": "Permiso para la corta, destrucción o descepado de formaciones xerofíticas según se establece en el artículo 151 del Reglamento del SEIA.",
            "fase": ["construcción"],
            "fuente": ["permiso ambiental sectorial"],
            "numero_fuente": ["203"],
            "objeto": "monitoreo, mantención, seguimiento y entrega de información",
            "sin_condiciones": "False",
            "frecuencia": "cumplimiento de permisos"
          },
          {
            "cell": "B12",
            "seccion": "pas",
            "numero_tabla": "6.1.1",
            "titulo": "En la fase de operación, la Dirección General de Aguas de la Región de Coquimbo otorgó el permiso correspondiente mediante el Ordinario N°22 del 14 de enero de 2021",
            "fase": ["operación"],
            "fuente": ["permiso ambiental sectorial"],
            "numero_fuente": [""],
            "objeto": "monitoreo, mantención, seguimiento y entrega de información",
            "sin_condiciones": "True",
            "frecuencia": "cumplimiento de permisos"
          },
        """, 
    "eg_output_1": """ 
          {
            "cell": "B12",
            "seccion": "pas",
            "numero_tabla": "6.1.1",
            "titulo": "En la fase de operación, la Dirección General de Aguas de la Región de Coquimbo otorgó el permiso correspondiente mediante el Ordinario N°22 del 14 de enero de 2021",
            "fase": ["operación"],
            "fuente": ["permiso ambiental sectorial"],
            "numero_fuente": ["151"],
            "objeto": "monitoreo, mantención, seguimiento y entrega de información",
            "sin_condiciones": "True",
            "frecuencia": "cumplimiento de permisos"
          },
          {
            "cell": "B12",
            "seccion": "pas",
            "numero_tabla": "6.1.1",
            "titulo": "Permiso para la corta, destrucción o descepado de formaciones xerofíticas según se establece en el artículo 151 del Reglamento del SEIA.",
            "fase": ["construcción"],
            "fuente": ["permiso ambiental sectorial"],
            "numero_fuente": ["203"],
            "objeto": "monitoreo, mantención, seguimiento y entrega de información",
            "sin_condiciones": "False",
            "frecuencia": "cumplimiento de permisos"
          }    
    """
}

### Funciones

In [ ]:
def gpt_5_nano(client, text, system_instructions_1, response_format_used, initial_messages,
                total_cost=0.0, total_input_tokens=0, total_output_tokens=0):
    
    # Armar la lista de mensajes incluyendo la instrucción del sistema y el mensaje del usuario
    messages = copy.deepcopy(initial_messages)
    messages.insert(0, {"role": "system", "content": system_instructions_1})
    messages.append({"role": "user", "content": text})
    # Solicitar la respuesta a la API
    completion = client.beta.chat.completions.parse(
        model="gpt-5-nano",
        response_format=response_format_used,
        messages=messages,
    )
    # Procesar la respuesta (se asume que el contenido es un JSON con la clave "steps")
    response_content = completion.choices[0].message.content
    response_dict = json.loads(response_content)
    df = pd.DataFrame(response_dict["steps"])
    tarifas = {
        'gpt-5-nano-2024-07-18': {
            'prompt': 0.00015,
            'completion': 0.00060
        },
    }
    modelo = completion.model
    tokens_prompt = completion.usage.prompt_tokens
    tokens_completion = completion.usage.completion_tokens
    if modelo not in tarifas:
        raise ValueError(f"Modelo no encontrado en tabla de tarifas: {modelo}")
    tarifa_prompt = tarifas[modelo]['prompt']
    tarifa_completion = tarifas[modelo]['completion']
    costo_prompt = (tokens_prompt / 1000) * tarifa_prompt
    costo_completion = (tokens_completion / 1000) * tarifa_completion
    costo_total = costo_prompt + costo_completion
    # Actualizar totales
    nuevo_total_cost = total_cost + costo_total
    nuevo_total_input_tokens = total_input_tokens + tokens_prompt
    nuevo_total_output_tokens = total_output_tokens + tokens_completion
    # Preparar diccionario con los totales actualizados
    totales_actualizados = {
        'total_cost': f"{nuevo_total_cost:.6f} USD",
        'total_input_tokens': nuevo_total_input_tokens,
        'total_output_tokens': nuevo_total_output_tokens,
        'detalle_uso_actual': {
            'modelo': modelo,
            'tokens_prompt': tokens_prompt,
            'tokens_completion': tokens_completion,
            'costo_prompt': f"{costo_prompt:.6f} USD",
            'costo_completion': f"{costo_completion:.6f} USD",
            'costo_total': f"{costo_total:.6f} USD"
        }
    }
    return df, totales_actualizados

def process_with_retries(client, df_chunk, section, initial_messages, 
                         total_cost=0.0, total_input_tokens=0, total_output_tokens=0, 
                         retries=3):
    for attempt in range(retries):
        try:
            current_df = df_chunk.copy()
            
            # Modificar el contenido según el número de intento
            if attempt == 1: 
                if 'contenido_variable' in current_df.columns:
                    current_df['contenido_variable'] = current_df['contenido_variable'].apply(
                        lambda x: x[:len(x)//3] if isinstance(x, str) else x
                    )
                    print("Segundo intento: Contenido reducido a un tercio")
            
            elif attempt == 2: 
                if 'contenido_variable' in current_df.columns:
                    current_df['contenido_variable'] = current_df['contenido_variable'].apply(
                        lambda x: x[:300] if isinstance(x, str) else x
                    )
                    print("Tercer intento: Contenido limitado a los primeros 1000 caracteres")
            
            # Convertir a JSON después de las modificaciones
            dfs_json = current_df.to_json(orient='records', force_ascii=False)
            df_aux, usage_info = gpt_5_nano(
                client, 
                dfs_json, 
                system_instructions[section]["categorizar"], 
                response_format[section], 
                initial_messages,
                total_cost,
                total_input_tokens,
                total_output_tokens
            )
            
            # Actualizar valores globales con los nuevos totales
            new_total_cost = float(usage_info['total_cost'].replace(' USD', ''))
            new_total_input_tokens = usage_info['total_input_tokens']
            new_total_output_tokens = usage_info['total_output_tokens']
            
            # Imprimir información sobre el uso actual
            current_usage = usage_info['detalle_uso_actual']
            print(f"Uso actual: {current_usage['tokens_prompt']} tokens prompt, "
                  f"{current_usage['tokens_completion']} tokens completion, "
                  f"costo: {current_usage['costo_total']}")
            print(f"Total acumulado: {usage_info['total_cost']}, "
                  f"{usage_info['total_input_tokens']} tokens prompt, "
                  f"{usage_info['total_output_tokens']} tokens completion")
            
            return df_aux, usage_info  # Retornar solo el DataFrame y el diccionario de uso
            
        except Exception as e:
            print(f"Error al procesar chunk en la sección {section}, intento {attempt + 1} de {retries}: {e}")
            time.sleep(2)  

    print(f"No se pudo procesar el chunk en la sección {section} después de {retries} intentos.")

    usage_info = {
        'total_cost': f"{total_cost:.6f} USD",
        'total_input_tokens': total_input_tokens,
        'total_output_tokens': total_output_tokens,
        'detalle_uso_actual': {
            'modelo': "error",
            'tokens_prompt': 0,
            'tokens_completion': 0,
            'costo_prompt': "0.000000 USD",
            'costo_completion': "0.000000 USD",
            'costo_total': "0.000000 USD"
        }
    }
    return pd.DataFrame(), usage_info  


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
def count_tokens(text):
    if not text or pd.isna(text):  
        return 0
    tokens = tokenizer.encode(text, add_special_tokens=False)
    return len(tokens)

### Formatos de Respuesta Pydantic

In [ ]:
class Seccion(str, Enum):
    pas = "pas"
    mitigacion = "mitigacion"
    plan_seguimiento = "plan_seguimiento"
    forma_cumplimiento = "forma_cumplimiento"
    compromisos_voluntarios = "compromisos_voluntarios"
    condiciones_exigencias = "condiciones_exigencias"
    contingencias_emergencias = "contingencias_emergencias"

class ComponenteMateria(str, Enum):
    aire = "aire"
    agua = "aguas"
    uso_suelos = "uso de suelos / urbanismo / vialidad"
    ruido = "ruidos"
    fauna = "fauna"
    flora = "flora"
    suelo = "terreno"
    paisaje = "paisaje"
    contaminacion_luminica = "contaminación lumínica"
    electricidad = "electricidad y combustibles"
    sustancias_peligrosas = "sustancias peligrosas"
    residuos_peligrosos = "residuos peligrosos"
    residuos_solidos = "residuos sólidos"
    residuos_liquidos = "residuos líquidos"
    patrimonio_cultural_arqueologico = "patrimonio cultural y arqueológico"
    particulares_mineria = f"temas propios del sector {sector}"
    social = "social y laboral"
 
class Fase(str, Enum):
    previo_a_la_construccion = "previo a la construcción"
    construccion = "construcción"
    cierre = "cierre"
    operacion = "operación"
    posterior_al_cierre = "posterior al cierre"

class Fuente(str, Enum):
    ley = "ley"
    decreto_supremo = "decreto supremo"
    decreto_ley = "decreto ley"
    decreto_fuerza_ley = "decreto con fuerza de ley"
    otra_norma = "otra norma"
    rca = "decisión de autoridad"
    cav = "Compromiso ambiental voluntario"
    pas = "permisos ambientales sectoriales"
    
class Objeto(str, Enum):
    monitoreo_informacion = "monitoreo, seguimiento y entrega de información"
    contingencias = "prevención de contingencias y respuesta ante emergencias"
    mitigacion_compensacion = "mitigación, compensación o reparación de efectos no deseados"
    cumplimiento_ambiental = "cumplimiento de normas ambientales"
    prevencion_ambiental = "prevención ambiental"
    otro = "otro"
    

class Independencia(str, Enum):
    principal = "Principal"
    secundaria = "Secundaria"

class Exigible(str, Enum):
    medios = "Obligación de medios"
    resultados = "Obligación de resultados" 

class Frecuencia(str, Enum):
    unica = "única"
    periodica = "periódica"
    permanente = "permanente"
    cumplimiento_permisos = "cumplimiento de permisos"
    en_caso_de_contingencia_o_emergencia = "eventual"


regex_fuentes = r"^\d+(?:/\d+)?$|^$"
regex = r"^(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])|(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])$|^$"
regex_pas = r"\d{1,3}"

descriptions = {
    "cell": f"Identificador único del documento revisado. Debe coincidir con el valor proporcionado en el input.",
    "seccion": f"Identificador de la sección del documento proporcionado en el input.",
    "numero_tabla": f"Identificador único de la tabla, que debe extraerse en el formato regex {regex}. Debe coincidir exactamente con el valor proporcionado.",
    "pas": f"Número de artículo del reglamento del SEIA, RSEIA o SEA correspondiente, que debe extraerse y registrarse en la variable 'pas' en el formato regex {regex_pas}. Si no existe, dejar vacío.",
    "normas":  f"Lista de normas, D.S (Decreto Supremo), y otros cuerpos legales, copiada directamente del texto original sin modificaciones. Si no existen, dejarla vacía.",
    "parte_asociada": f"Emplazamiento, parte, obra, acción, emisión, residuo o sustancia a las que aplica la obligación. Registrar tal cual aparece en el texto original.",
    "fase": f"Fases del proyecto a las que aplica la obligación, registradas exactamente como en el texto original. Si se menciona de forma genérica ('todas las fases', 'durante la vida útil'), se debe posteriormente categorizar en ['construcción','operación','cierre'].",
    "componente_materia": f"Texto original que describe el componente o materia ambiental abordado. Copiar fielmente del documento fuente.",
    "control_seguimiento": f"Métodos, procedimientos o acciones específicas (incluyendo registros, mediciones, inspecciones) para controlar o hacer seguimiento del cumplimiento de la obligación. Copiar fielmente del texto original.",
    "condiciones": f"Condiciones o exigencias específicas para el otorgamiento registradas tal cual en el texto original. Si no existen, la variable 'sin_condiciones' debe marcarse como True.",
    "forma_cumplimiento": f"Detalle de la forma de cumplimiento, incluyendo métodos, equipos o acciones para garantizar el cumplimiento. Copiar directamente del texto original.",
    "acciones_contingencia": f"Acciones o medidas a implementar para prevenir o enfrentar una contingencia. Copiar directamente del texto original. Aplica solo si se trata de contingencias o emergencias.",
    "acciones_emergencia": f"Acciones o medidas a implementar para controlar una emergencia. Copiar directamente del texto original. Aplica solo si se trata de contingencias o emergencias.",
    "riesgo_contingencia": f"Riesgo o contingencia descrita en el texto original. Copiar fielmente. Aplica solo si se trata de contingencias o emergencias.",
    "indicador_cumplimiento": f"Indicador o parámetro que permite evaluar el cumplimiento de la obligación. Copiar directamente del texto original.",
    "objetivo_descripcion_justificacion": f"Objetivo, descripción y justificación de la obligación, tal como está en el texto original.",
    "lugar_forma_oportunidad": f"Lugar, forma y oportunidad (momento o condiciones temporales) de implementación de la obligación. Copiar directamente del texto original.",
    "impacto":  f"Impacto asociado a la obligación, copiado directamente del texto original.",
    "mitigacion": f"Medida o medidas asociadas a la mitigación del impacto descrito, copiadas directamente del texto original.",
    "resumen": f"Breve resumen de la obligación, interpretado por el modelo, que describa claramente la acción o exigencia principal.",
    "categoria_componente_materia": f"Categoría temática asignada a la materia ambiental protegida. Por ejemplo: 'aire', 'suelo', 'residuos líquidos'. Debe ser coherente con el componente identificado en 'componente_materia'.",
    "categoria_fase": f"Lista de fases del proyecto a las que aplica la obligación, a partir del texto en 'fase'. Si 'fase' indica todas las etapas o la vida útil completa, clasificar como ['construcción', 'operación', 'cierre'].",
    "fuente": f"Lista de las fuentes legales o reglamentarias mencionadas. Las abreviaciones deben ampliarse: 'D.S.' = 'decreto supremo', 'R.Ex.' = 'resolución', 'd.f.l' = 'decreto con fuerza de ley', 'reglamento' = 'decreto supremo'. Si no hay normas y 'normas' está vacío, usar ['decisión de autoridad']. Si 'seccion' = 'compromisos_voluntarios', usar ['compromiso ambiental voluntario'].",
    "numero_fuente": f"Lista de los identificadores correspondientes a cada fuente legal mencionada, preservando el orden. Si no hay normas específicas mencionadas o si 'fuente' contiene una decisión de autoridad o compromiso ambiental voluntario usar [''] como valor. Si 'seccion' es 'pas', el valor siempre debe ser el número del PAS (entre 111 y 160).",
    "objeto": f"Objetivo principal que la obligación busca lograr o proteger, interpretado del contexto del texto original.Se deben priorizar otras categorías por sobre 'Cumplimiento de Normas Ambientales', y esta categoría se prioriza por sobre 'Otros'.",
    "independencia": f"Grado de independencia de la obligación: 'principal' si describe una medida central exigida (por ejemplo, 'Implementar un plan de reducción de emisiones'); 'secundaria' si su función es asegurar, verificar, registrar, informar o demostrar el cumplimiento de otra obligación principal (por ejemplo, 'Mantener registro de retiro y disposición final de residuos', 'Verificar el cumplimiento', 'Reporte fotográfico')",
    "exigible": f"Resultado exigible de la obligación: 'obligación de medios' si exige implementar acciones, métodos o programas para alcanzar un resultado (ej. 'Implementar un programa de control periódico'), y 'obligación de resultados' si exige alcanzar un resultado específico y medible (ej. 'Mantener las emisiones de CO2 por debajo de X valor'). La distinción es que las obligaciones de medios establecen cómo actuar, mientras que las de resultados exigen un estándar final que debe cumplirse.",
    "justificacion_exigible": f"Extracto del texto original que justifica la obligación. Debe copiarse tal cual aparezca en la fuente.",
    "frecuencia": f"Categoría que describe la periodicidad con la que debe cumplirse la obligación. Si se menciona 'anual', 'semestral', 'mensual', 'semanal' u otra frecuencia específica, clasificarla como 'periódica'. Si no se menciona, dejarla vacía.",
    "sin_condiciones": f"Booleano. True si no existen condiciones adicionales en la tabla, False en caso contrario.",
    "titulo": f"Título de la tabla. Si no es evidente, crearlo en base al contenido y reflejarlo en el resultado final."
}


class UNICO(BaseModel):
    # Variables Originales
    cell: str = Field(description=descriptions["cell"])
    seccion: Seccion = Field(description=descriptions["seccion"])
    numero_tabla: str = Field(description=descriptions["numero_tabla"])
    # Categorías Nuevas
    titulo: str = Field(description=descriptions["titulo"])
    fase: List[Fase] = Field(description=descriptions["categoria_fase"])
    fuente: List[Fuente] = Field(description=descriptions["fuente"])
    numero_fuente: List[str] = Field(description=descriptions["numero_fuente"])
    objeto: Objeto = Field(description=descriptions["objeto"])
    sin_condiciones: bool = Field(description=descriptions["sin_condiciones"])

class MULTIPLE(BaseModel):
    # Variables Originales
    cell: str = Field(description=descriptions["cell"])
    seccion: Seccion = Field(description=descriptions["seccion"])
    numero_tabla: str = Field(description=descriptions["numero_tabla"])
    # Categorías Nuevas
    resumen: str = Field(description=descriptions["resumen"])
    componente_materia: ComponenteMateria = Field(description=descriptions["categoria_componente_materia"])
    fase: List[Fase] = Field(description=descriptions["categoria_fase"])
    fuente: List[Fuente] = Field(description=descriptions["fuente"])
    numero_fuente: List[str] = Field(description=descriptions["numero_fuente"])
    objeto: Objeto = Field(description=descriptions["objeto"])
    orden_obligacion: Independencia = Field(description=descriptions["independencia"])
    tipo_obligacion: Exigible = Field(description=descriptions["exigible"])
    justificacion: str = Field(description=descriptions["justificacion_exigible"])
    frecuencia: Frecuencia = Field(description=descriptions["frecuencia"])


class Multiple(BaseModel):
    steps: List[MULTIPLE]

class Unico(BaseModel):
    steps: List[UNICO]
    
response_format = {
    "mitigacion": Multiple,
    "pas": Unico,
    "plan_seguimiento": Multiple, 
    "forma_cumplimiento": Multiple,
    "compromisos_voluntarios": Multiple,
    "condiciones_exigencias": Multiple,
    "contingencias_emergencias": Unico,
}

system_instructions = {
    "mitigacion": multiple,
    "pas": unico,
    "plan_seguimiento": multiple, 
    "forma_cumplimiento": multiple,
    "compromisos_voluntarios": multiple,
    "condiciones_exigencias": multiple,
    "contingencias_emergencias": unico,
}


duplicacion_instructions = {
    "mitigacion": duplicacion_multiple,
    "pas": duplicacion_unico,
    "plan_seguimiento": duplicacion_multiple, 
    "forma_cumplimiento": duplicacion_multiple,
    "compromisos_voluntarios": duplicacion_multiple,
    "condiciones_exigencias": duplicacion_multiple,
    "contingencias_emergencias": duplicacion_unico,
}

### Importar Archivo Base

In [ ]:
df = pd.read_excel(os.path.join(direccion_output, "df_subconsiderandos.xlsx" ))
dfs = df
dfs.fillna("", inplace=True)

In [ ]:
# Ruta del archivo Excel
file_path = os.path.join(direccion_output, f"seia_{sector}.xlsx")
dfz = pd.read_excel(file_path, engine='openpyxl')
workbook = openpyxl.load_workbook(file_path, data_only=True)
sheet = workbook.active

# Función para extraer hipervínculos de una celda
def extract_hyperlinks(sheet):
    hyperlinks = {}
    for row in sheet.iter_rows():
        for cell in row:
            if cell.hyperlink:
                hyperlinks[cell.coordinate] = cell.hyperlink.target
    return hyperlinks

# Obtener los hipervínculos del archivo
hyperlinks = extract_hyperlinks(sheet)

# Crear nuevas columnas "cell" y "hyperlink" en el DataFrame
dfz["cell"] = ""
dfz["hyperlink"] = ""

# Asignar las coordenadas de las celdas y los hipervínculos a las columnas correspondientes
for i, (cell, hyperlink) in enumerate(hyperlinks.items()):
    if i < len(dfz):
        dfz.at[i, "hyperlink"] = hyperlink


### Procesar Archivo Base

In [ ]:
# modificar filas donde 'numero_tabla' es "no identificado"
dfs['numero_tabla'] = dfs['numero_tabla'].astype(str)
dfs['numero_tabla'] = dfs['numero_tabla'].str.replace("no.identificado", "no identificado")
mask = dfs['numero_tabla'] == "no identificado"
dfs.loc[mask, 'numero_tabla'] = dfs.loc[mask].groupby(['seccion', 'cell']).cumcount().add(1).astype(str).radd("no identificado ")

# Ahora aplicar drop_duplicates para eliminar duplicados en las columnas 'numero_tabla' y 'cell'
dfs['tokens'] = dfs.apply(lambda row: count_tokens(row.to_json(force_ascii=False)), axis=1)
dfs = dfs.sort_values(by='tokens', ascending=False)
dfs = dfs.drop_duplicates(subset=['numero_tabla', 'cell'])
dfs = dfs.drop(columns=['tokens'])
dfs = dfs.sort_values(by=['cell', 'seccion', 'numero_tabla'], ascending=True)

# Variables a transformar en formato long
variables_long = [
    "forma_cumplimiento", "indicador_cumplimiento", "lugar_forma_oportunidad",
    "control_seguimiento", "condiciones", "objetivo_descripcion_justificacion",
    "acciones_emergencia", "acciones_contingencia"
]

# Variables a mantener en formato wide
variables_wide = [
    "numero_tabla", "seccion", "pas", "normas", "fase", "parte_asociada", "componente_materia",
    "riesgo_contingencia", "impacto", "mitigacion", "cell"
]

# Transformar el DataFrame al formato long
dfs = dfs.melt(
    id_vars=variables_wide,
    value_vars=variables_long,
    var_name="nombre_variable",
    value_name="contenido_variable"
)
dfs = dfs[dfs["contenido_variable"].str.strip() != ""]
dfs = dfs[dfs["contenido_variable"].str.strip() != "0"]
dfs = dfs[dfs["contenido_variable"].str.strip() != "-"]
dfs = dfs[dfs["contenido_variable"].str.strip() != "----"]
dfs.loc[dfs['nombre_variable'].isin(['indicador_cumplimiento', 'control_seguimiento']), ['impacto', 'mitigacion']] = ""
dfs = dfs[~(dfs["contenido_variable"].str.contains(r"Referencia al ICE|No se especifica|No posee.|No se considera.|Registros.|del ICE|Informe Consolidado de Evaluación|RCA del Proyecto\.|Referencia a documentos del expediente|Referencia a documentos del Anexo|Referencia al expediente|Referencia del EIA|Adenda Complementaria de la DIA|Ver numeral \d+(?:\.\d+)* del ICE", na=False) & dfs["nombre_variable"].isin(["control_seguimiento", "indicador_cumplimiento"]))]

### Código Principal

In [ ]:
df_obligaciones = pd.DataFrame()
df_final = pd.DataFrame()

# Variables para seguimiento de costos y tokens
total_cost = 0.0
total_input_tokens = 0
total_output_tokens = 0

for c in dfs.cell.unique()[:]:
    if c in dfz.id.unique()[:]: 
        print(f"\nProcesando archivo: {c}")    
        dfs_aux = dfs[dfs.cell == c].copy()
        df_obligaciones = pd.DataFrame()
        
        # Iterar sobre cada sección de las instrucciones del sistema
        for section in system_instructions.keys():
            try:
                # Dividir la sección en chunks
                dfs_section = dfs_aux[dfs_aux.seccion == section].copy()
    
                # Convertir NaNs a cadenas vacías
                dfs_section = dfs_section.fillna("")
                
                # Crear columna de tokens y verificar si se creó correctamente
                dfs_section['tokens'] = dfs_section.apply(lambda row: count_tokens(row.to_json(force_ascii=False)), axis=1)
                
                if 'tokens' not in dfs_section.columns:
                    print(f"La columna 'tokens' no se creó correctamente en la sección {section}. Saltando...")
                    continue
    
                section_chunks = []
                current_chunk = []
                current_tokens = 0
        
                # Dividir en chunks basados en el límite de tokens
                for idx, row in dfs_section.iterrows():
                    row_tokens = row['tokens']
                    limit = 4000 if section not in ["pas", "contingencias_emergencias"] else 2500
    
                    # Verificar si añadir la fila actual supera el límite de tokens
                    if current_tokens + row_tokens > limit:
                        section_chunks.append(pd.DataFrame(current_chunk))
                        current_chunk = []  # Reiniciar chunk
                        current_tokens = 0
        
                    current_chunk.append(row)  # Añadir la fila actual al chunk
                    current_tokens += row_tokens
    
                # Agregar el último chunk si queda algo
                if current_chunk:
                    section_chunks.append(pd.DataFrame(current_chunk))
        
                # Procesar cada chunk por separado
                for i, chunk in enumerate(section_chunks, 1):
                    if chunk.empty:
                        print(f"El chunk {i} de la sección {section} está vacío. Saltando...")
                        continue
                    
                    if 'tokens' not in chunk.columns:
                        print(f"El chunk {i} de la sección {section} no contiene la columna 'tokens'. Saltando...")
                        continue
    
                    # Obtener el número total de tokens en el chunk
                    current_tokens = chunk['tokens'].sum()
                    print(f"Procesando chunk {i} de la sección {section}, tokens: {current_tokens}")
              
                    # Definir mensajes iniciales
                    initial_messages = [
                        {"role": "user", "content": system_instructions[section]['eg_input_1']},
                        {"role": "assistant", "content": system_instructions[section]['eg_output_1']},
                        {"role": "user", "content": system_instructions[section]['eg_input_2']},
                        {"role": "assistant", "content": system_instructions[section]['eg_output_2']}
                    ]
        
                    # Ejecutar con reintentos - Ahora pasamos el DataFrame en lugar del JSON
                    df_aux, usage_info = process_with_retries(
                        client, 
                        chunk, 
                        section, 
                        initial_messages,
                        total_cost,
                        total_input_tokens,
                        total_output_tokens
                    )
    
                    # Concatenar el resultado al DataFrame final
                    if df_aux is not None:
                        df_obligaciones = pd.concat([df_obligaciones, df_aux], ignore_index=True)
            
            except Exception as e:
                print(f"Error procesando la sección {section}: {e}")
                traceback.print_exc()
    
        ######--------------------------------------------------------------------------------------------------------------------
        # FORMATEAR
        dff = df_obligaciones[df_obligaciones.cell == str(c)].copy()
        print("formatear")
        
        # Agrupar por 'numero_tabla'
        dff.sort_values(by='numero_tabla').reset_index(drop=True)
        grouped = dff.groupby('numero_tabla')
    
        regex = r"^(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])|(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])$|^$"
        for numero_tabla, group in grouped:
            if len(group) > 1:  
                try: 
                    section = group.seccion.iloc[0]
                    
                    # Ejecutar la función GPT para categorizar y formatear
                    initial_messages = [
                        {"role": "user", "content": f"{duplicacion_instructions[section][f'eg_input_1']}"},
                        {"role": "assistant", "content": f"{duplicacion_instructions[section][f'eg_output_1']}"},
                    ]
                    df_aux, usage_info = process_with_retries(
                        client,  
                        group,  
                        section, 
                        initial_messages,
                        total_cost,
                        total_input_tokens,
                        total_output_tokens
                    )
    
                    if len(group) - len(df_aux) > 0: 
                        print(f"Eliminadas {len(group) - len(df_aux)} filas en numero_tabla {numero_tabla}: {section}")
                    else: 
                        print(f"se intentó eliminar filas, pero no se eliminaron en {numero_tabla}: {section}")
    
                    if (section in ["pas", "contingencias_emergencias"]) and len(df_aux) > 1: 
                        try:
                            longest_row = group.loc[group.apply(lambda row: len(''.join(row.astype(str))), axis=1).idxmax()]
                            df_final = pd.concat([df_final, pd.DataFrame([longest_row])], ignore_index=True)
                        except Exception as e:
                            print(f"Error al procesar eliminaciones en numero_tabla {numero_tabla}: {e}")
                    
                    # Concatenar el resultado al DataFrame final
                    df_final = pd.concat([df_final, df_aux], ignore_index=True)
                    print("concatenado")
        
                except Exception as e:
                    print(f"error: {e}")
                    try: 
                        print(f"Reintentando Proceso {len(group)} filas con numero_tabla {numero_tabla}")
                        
                        # Modificación para usar la nueva función de reintentos
                        df_aux, usage_info = process_with_retries(
                            client,  
                            group,  
                            section, 
                            initial_messages,
                            total_cost,
                            total_input_tokens,
                            total_output_tokens
                        )
    
                        
                        if len(group) - len(df_aux) > 0: 
                            print(f"Eliminadas {len(group) - len(df_aux)} filas en numero_tabla {numero_tabla}: {section}")
                        else: 
                            print(f"se intentó eliminar filas, pero no se eliminaron en {numero_tabla}: {section}")
    
                        if (section in ["pas", "contingencias_emergencias"]) and len(df_aux) > 1: 
                            try:
                                longest_row = group.loc[group.apply(lambda row: len(''.join(row.astype(str))), axis=1).idxmax()]
                                df_final = pd.concat([df_final, pd.DataFrame([longest_row])], ignore_index=True)
                            except Exception as e:
                                print(f"Error al procesar eliminaciones en numero_tabla {numero_tabla}: {e}")
                        
                        df_final = pd.concat([df_final, df_aux], ignore_index=True)
                        print("concatenado")
                        
                    except Exception as e:
                        # Find the longest row by character length and add it to df_final
                        print(f"error nuevamente: {e}")
                        longest_row = group.loc[group.apply(lambda row: len(''.join(row.astype(str))), axis=1).idxmax()]
                        df_final = pd.concat([df_final, pd.DataFrame([longest_row])], ignore_index=True)
                        print("concatenado")
            
            else:
                print(f"Numero_tabla {numero_tabla} no tiene duplicados, no se procesa.")
                df_final = pd.concat([df_final, group], ignore_index=True)
                print("concatenado")
    
            df_final.to_excel(os.path.join(direccion_output, f'obligaciones.xlsx'), index=False)

# Imprimir resumen final de costos y tokens
print("\n========== RESUMEN FINAL DE USO ==========")
print(f"Total de tokens de entrada: {total_input_tokens}")
print(f"Total de tokens de salida: {total_output_tokens}")
print(f"Total de tokens: {total_input_tokens + total_output_tokens}")
print(f"Costo total estimado: ${total_cost:.6f} USD")
print("===========================================")

df_final.to_excel(os.path.join(direccion_output, f'df_obligaciones.xlsx'), index=False)